Import Required Packages

In [9]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

Import the csv file as data

In [10]:
df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 2 Files\price_response.csv")

In [11]:
df

,Product,Intercept,Sensitivity,Capacity
0,Line 1 Product 1,35234.545786,-45.896450,80020.0
1,Line 1 Product 2,37790.240832,-8.227794,89666.0
2,Line 1 Product 3,35675.333217,-7.584436,80638.0
3,Line 2 Product 1,37041.380378,-9.033166,86740.0
4,Line 2 Product 2,36846.140386,-4.427869,84050.0
5,Line 2 Product 3,35827.023747,-2.629060,86565.0
6,Line 3 Product 1,39414.266325,-2.421484,87051.0
7,Line 3 Product 2,35991.951460,-4.000512,85156.0
8,Line 3 Product 3,39313.317031,-2.296622,87588.0


# (a)

When TechEssentials initially launched 15 years ago, it had just one product line (the EvoTech
Pro Series computers) with two versions: Basic and Advanced. Apply the KKT conditions to
find the optimal prices of each of the two computers within this series, taking into account that
the Advanced version should have a higher price as compared to the Basic one. That is,

Maximize p1 (a1 − b1p1) + p2 (a1 − b1p2)

for some prices pi ≥ 0 and linear price response function coefficients ai ≥ 0 and bi ≥ 0 where
i = {1, 2} subject to price ordering and demand non-negativity constraints.

In [12]:
# Extract parameters from the dataframe
a1 = df.loc[df['Product'] == 'Line 1 Product 1', 'Intercept'].values[0]
b1 = abs(df.loc[df['Product'] == 'Line 1 Product 1', 'Sensitivity'].values[0])

# Create a new model
m = gp.Model("pricing_optimization")

# Decision variables
p1 = m.addVar(vtype=GRB.CONTINUOUS, name="p1")  # Price of Basic version
p2 = m.addVar(vtype=GRB.CONTINUOUS, name="p2")  # Price of Advanced version

# Objective function
m.setObjective(p1 * (a1 - b1 * p1) + p2 * (a1 - b1 * p2), GRB.MAXIMIZE)

# Constraint: Price ordering (p2 > p1)
m.addConstr(p2 - p1 >= 0, "price_ordering")

# Constraints: Demand non-negativity
m.addConstr(a1 - b1 * p1 >= 0, "demand_non_negativity_basic")
m.addConstr(a1 - b1 * p2 >= 0, "demand_non_negativity_advanced")

# Optimize model
m.optimize()

# Print results
print("Optimal prices:")
print("Basic version price:", p1.x)
print("Advanced version price:", p2.x)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x888c97ca
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [9e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 4e+04]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time

# (b) 

Contrast the solution obtained from the previous question with the solution obtained from the
application of a projected gradient descent algorithm. Note that the constraint set is linear, so
the projection step can be solved as a quadratic program. Start with all prices initialized to zero,
a step size of 0.001, and a stopping criterion of 10−6
, what do you get as the optimal prices?


In [14]:
# Define objective function
def objective_function(prices):
    p1, p2 = prices
    return -(p1 * (a1 - b1 * p1) + p2 * (a1 - b1 * p2))

# Define constraint function for projecting onto the feasible set
def constraint_function(prices):
    return prices[0] - prices[1]

# Define projection function
def projection_function(prices):
    p1_proj = max(0, prices[0])
    p2_proj = max(0, prices[1])
    return [p1_proj, p2_proj]

# Define projected gradient descent algorithm
def projected_gradient_descent(init_prices, step_size, stopping_criterion):
    prices = np.array(init_prices, dtype=np.float64)  # Ensure float64 data type
    while True:
        # Compute gradient of the objective function
        gradient = np.array([-a1 + 2 * b1 * prices[0] + b1 * prices[1], -a1 + 2 * b1 * prices[1] + b1 * prices[0]])
        
        # Update prices
        prices -= step_size * gradient
        
        # Project onto feasible set
        prices = projection_function(prices)
        
        # Check stopping criterion
        if np.linalg.norm(gradient) < stopping_criterion:
            break
    
    return prices

# Define initial prices, step size, and stopping criterion
initial_prices = [0, 0]
step_size = 0.001
stopping_criterion = 1e-6

# Run projected gradient descent algorithm
optimal_prices_pgd = projected_gradient_descent(initial_prices, step_size, stopping_criterion)

# Print optimal prices
print("Optimal Prices (Projected Gradient Descent):")
print("Basic EvoTech Laptop Price:", optimal_prices_pgd[0])
print("Advanced EvoTech Laptop Price:", optimal_prices_pgd[1])

Optimal Prices (Projected Gradient Descent):
Basic EvoTech Laptop Price: 255.89884773493702
Advanced EvoTech Laptop Price: 255.89884773493702


# (c) 

Now consider all three product lines and the three versions within each product line. Assuming
that the Basic, Advanced, and Premium versions of the product should be increasing in price
within each product line, formulate and solve a quadratic optimization problem in Gurobi. What
is the optimal revenue suggested by the model?

In [15]:
# Data
product_lines = ['EvoTech Pro Series', 'InfiniteEdge Notebooks', 'FusionBook Elite Series']
versions = ['Basic', 'Advanced', 'Premium']

# Price response function coefficients (taken from the provided CSV)
coefficients = {
    ('EvoTech Pro Series', 'Basic'): (35234.54579, -45.89644971),
    ('EvoTech Pro Series', 'Advanced'): (37790.24083, -8.227794173),
    ('EvoTech Pro Series', 'Premium'): (35675.33322, -7.58443641),
    ('InfiniteEdge Notebooks', 'Basic'): (37041.38038, -9.033166404),
    ('InfiniteEdge Notebooks', 'Advanced'): (36846.14039, -4.427869206),
    ('InfiniteEdge Notebooks', 'Premium'): (35827.02375, -2.629060015),
    ('FusionBook Elite Series', 'Basic'): (39414.26632, -2.421483918),
    ('FusionBook Elite Series', 'Advanced'): (35991.95146, -4.000512401),
    ('FusionBook Elite Series', 'Premium'): (39313.31703, -2.296622373)
}

# Create a new Gurobi model
model = gp.Model("Optimal_Pricing")

# Decision variables
prices = {}
for line in product_lines:
    for version in versions:
        prices[line, version] = model.addVar(vtype=GRB.CONTINUOUS, name=f"price_{line}_{version}", lb=0)

# Set objective function: maximize total revenue
total_revenue = sum(prices[line, version] * (coefficients[line, version][0] - coefficients[line, version][1] * prices[line, version])
                    for line in product_lines for version in versions)
model.setObjective(total_revenue, GRB.MAXIMIZE)

# Add constraints: prices should increase within each product line
for line in product_lines:
    for i in range(len(versions) - 1):
        model.addConstr(prices[line, versions[i]] <= prices[line, versions[i + 1]], name=f"price_ordering_{line}_{i}")

# Optimize model
model.optimize()

# Print optimal prices and revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for line in product_lines:
        for version in versions:
            print(f"{line} - {version}:", prices[line, version].x)
    print("Optimal Revenue:", model.objVal)
else:
    print("Optimization did not converge to an optimal solution.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 6 rows, 9 columns and 12 nonzeros
Model fingerprint: 0x89184ed4
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 5.004070e+19
Presolve time: 0.06s
Presolved: 16 rows, 20 columns, 40 nonzeros
Presolved model has 9 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.

Va

# (d) 

Considering that prices should increase (i) within each product line (Basic, Advanced, Premium), and (ii) within the same version (e.g., a FusionBook Elite should be more expensive
than an InfiniteEdge Notebook which should be more expensive than an EvoTech Pro), modify
the previous quadratic optimization problem by adding new constraints to ensure this logic is
captured in the model. What is the optimal revenue TechEssentials can anticipate now?

In [17]:
# Data
product_lines = ['EvoTech Pro Series', 'InfiniteEdge Notebooks', 'FusionBook Elite Series']
versions = ['Basic', 'Advanced', 'Premium']

# Price response function coefficients (taken from the provided CSV)
coefficients = {
    ('EvoTech Pro Series', 'Basic'): (35234.54579, -45.89644971),
    ('EvoTech Pro Series', 'Advanced'): (37790.24083, -8.227794173),
    ('EvoTech Pro Series', 'Premium'): (35675.33322, -7.58443641),
    ('InfiniteEdge Notebooks', 'Basic'): (37041.38038, -9.033166404),
    ('InfiniteEdge Notebooks', 'Advanced'): (36846.14039, -4.427869206),
    ('InfiniteEdge Notebooks', 'Premium'): (35827.02375, -2.629060015),
    ('FusionBook Elite Series', 'Basic'): (39414.26632, -2.421483918),
    ('FusionBook Elite Series', 'Advanced'): (35991.95146, -4.000512401),
    ('FusionBook Elite Series', 'Premium'): (39313.31703, -2.296622373)
}

# Create a new Gurobi model
model = gp.Model("Optimal_Pricing")

# Decision variables
prices = {}
for line in product_lines:
    for version in versions:
        prices[line, version] = model.addVar(vtype=GRB.CONTINUOUS, name=f"price_{line}_{version}", lb=0)

# Set objective function: maximize total revenue
total_revenue = sum(prices[line, version] * (coefficients[line, version][0] - coefficients[line, version][1] * prices[line, version])
                    for line in product_lines for version in versions)
model.setObjective(total_revenue, GRB.MAXIMIZE)

# Add constraints: prices should increase within each product line
for line in product_lines:
    for i in range(len(versions) - 1):
        model.addConstr(prices[line, versions[i]] <= prices[line, versions[i + 1]], name=f"price_ordering_{line}_{i}")

# Add constraints: prices should increase across different product lines
for i in range(len(product_lines) - 1):
    for version in versions:
        model.addConstr(prices[product_lines[i], version] <= prices[product_lines[i + 1], version], name=f"price_ordering_across_{version}")

# Optimize model
model.optimize()

# Print optimal prices and revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for line in product_lines:
        for version in versions:
            print(f"{line} - {version}:", prices[line, version].x)
    print("Optimal Revenue:", model.objVal)
else:
    print("Optimization did not converge to an optimal solution.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 12 rows, 9 columns and 24 nonzeros
Model fingerprint: 0xa9febee9
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.186568e+18
Presolve time: 0.00s
Presolved: 22 rows, 20 columns, 52 nonzeros
Presolved model has 9 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.

F

# (e) 

What set of prices, from (c) or (d), do you think makes the most sense to implement in practice?

# (f) 

Do you believe something is missing such that the model does not faithfully represent reality.